In [99]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Load the CSV file
df = pd.read_csv('validator_data.csv')

# Function to create lag features
def create_lag_features(df, lag=1):
    for i in range(1, lag + 1):
        df[f'lag_{i}'] = df['effectiveness'].shift(i)
    return df

# Create lag features for the past 5 days
df = df.groupby('id').apply(create_lag_features, lag=1)
df = df.dropna().reset_index(drop=True)
# print(df.head())

# Define features and target
# features = [col for col in df.columns if 'lag_' in col]
features = ['total_reward']
target = 'effectiveness'

# Split the data into training and testing sets
train_df = df[df['partition_date'] < '2024-05-02']
test_df = df[df['partition_date'] >= '2024-05-02']

X_train = train_df[features]
y_train = train_df[target]
X_test = test_df[features]
y_test = test_df[target]

# Initialize and train the XGBoost model
model = XGBRegressor()
model.fit(X_train.values, y_train.values)

print(type(X_test.values), type(df), X_test.values.shape, X_train.values.shape)

# Make predictions
test_df['predicted_change'] = model.predict(X_test.values)

print(model.predict(X_test.values[:1]))

# Evaluate the model
mse = mean_squared_error(y_test, test_df['predicted_change'])
print(f'Mean Squared Error: {mse}')

# Select the stock with the highest predicted change for tomorrow
investment_stock = test_df.loc[test_df['predicted_change'].idxmax()]
print(f'Stock to invest in: {investment_stock["id"]}, Predicted Change: {investment_stock["predicted_change"]}')


<class 'numpy.ndarray'> <class 'pandas.core.frame.DataFrame'> (1237, 1) (3883, 1)
[0.9743774]
Mean Squared Error: 0.00023868440270298732
Stock to invest in: Chorus One, Predicted Change: 0.9825234413146973


/var/folders/pj/vyqltmgj79xghgpb38_96m040000gn/T/ipykernel_98715/1031454150.py:17: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df = df.groupby('id').apply(create_lag_features, lag=1)
/var/folders/pj/vyqltmgj79xghgpb38_96m040000gn/T/ipykernel_98715/1031454150.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['predicted_change'] = model.predict(X_test.values)


In [147]:
test_df.values

array([['network', 290631, 2063, ..., '2024-05-02', 0.9806428592230352,
        0.9743773937225342],
       ['Launchnodes', 2562, 15, ..., '2024-05-02', 0.8364000474332984,
        0.9464616775512695],
       ['CryptoManufaktur', 8722, 53, ..., '2024-05-02',
        0.9830070044910248, 0.9662720561027527],
       ...,
       ['Attestant (BVI) Limited', 8892, 48, ..., '2024-06-02',
        0.9869107547569268, 0.9673405289649963],
       ['Develp GmbH', 5500, 44, ..., '2024-06-02', 0.9851771155105016,
        0.9807904362678528],
       ['Gateway.fm AS', 4500, 28, ..., '2024-06-02', 0.9754852924909982,
        0.965843915939331]], dtype=object)

In [100]:
!giza users create-api-key

[giza][2024-06-09 13:05:13.700] Creating API Key ✅ 
[giza][2024-06-09 13:05:14.872] Successfully created API Key. It will be used for future requests ✅


In [101]:
import sklearn
print(sklearn.__version__)

1.4.2


In [102]:
from giza.zkcook import serialize_model
serialize_model(model, "validator_data.json")

In [103]:
! giza transpile validator_data.json --output-path validator_data

[giza][2024-06-09 13:05:22.239] No model id provided, checking if model exists ✅
[giza][2024-06-09 13:05:22.241] Model name is: validator_data
[giza][2024-06-09 13:05:23.097] Model already exists, using existing model ✅ 
[giza][2024-06-09 13:05:23.097] Model found with id -> 761! ✅
[giza][2024-06-09 13:05:24.020] Version Created with id -> 3! ✅
[giza][2024-06-09 13:05:24.021] Sending model for transpilation ✅ 
[giza][2024-06-09 13:05:50.849] Transpilation is fully compatible. Version compiled and Sierra is saved at Giza ✅
⠧ Transpiling Model...
[giza][2024-06-09 13:05:53.409] Downloading model ✅
[giza][2024-06-09 13:05:53.413] model saved at: validator_data


In [104]:
! giza endpoints deploy --size M --model-id 761 --version-id 3

▰▱▱▱▱▱▱ Creating endpoint!t!
[giza][2024-06-09 13:06:13.583] Endpoint is successful ✅
[giza][2024-06-09 13:06:13.586] Endpoint created with id -> 331 ✅
[giza][2024-06-09 13:06:13.586] Endpoint created with endpoint URL: https://endpoint-ashu-761-3-aa24c0d5-7i3yxzspbq-ew.a.run.app 🎉


In [143]:
import requests
import json

url = "https://endpoint-ashu-761-3-aa24c0d5-7i3yxzspbq-ew.a.run.app/cairo_run"

payload = json.dumps({
  "args": "[538000 7459999 7459999 7459999 7459999 7459999]"
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)


{"result":"94390","request_id":"190dcaea388d4ca59a497a600ad95c74"}


In [106]:
print(X_test.values[0].shape)

(1,)


In [105]:
v = X_test.values[:1]
args = v[0]*100000
int_args = [int(arg) for arg in args]
print(int_args)

[87545018]


In [107]:
X_train.iloc[0]

total_reward    36.010773
Name: 0, dtype: float64

In [134]:
new_data = pd.DataFrame({
    'total_reward': [4564, 5345]
})
new_data
# model.predict(new_data.values)

,total_reward
0,4564
1,5345


In [145]:
import xgboost as xgb
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from giza.agents.model import GizaModel

MODEL_ID = 761  # Update with your model ID
VERSION_ID = 3  # Update with your version ID

def prediction(input, model_id, version_id):
    model = GizaModel(id=model_id, version=version_id)

    (result, proof_id) = model.predict(
        input_feed={"input": input}, verifiable=True, model_category="XGB"
    )

    return result, proof_id


def execution():
    # The input data type should match the model's expected input
    # input = X_test.values[:1]
    # print(model.predict(input))
    # input = new_data.astype(np.float32).values
    input = np.array([[36.0107], [43.5353]])
    print(input.flatten(), input.shape)

    (result, proof_id) = prediction(input, MODEL_ID, VERSION_ID)

    print(f"Predicted value for input {input.flatten()[0]} is {result}")

    return result, proof_id


if __name__ == "__main__":
    _, proof_id = execution()
    print(f"Proof ID: {proof_id}")

[36.0107 43.5353] (2, 1)


An error occurred in predict: 503 Server Error: Service Unavailable for url: https://endpoint-ashu-761-3-aa24c0d5-7i3yxzspbq-ew.a.run.app/cairo_run
Deployment predict error: Service Unavailable
An error occurred in predict: 503 Server Error: Service Unavailable for url: https://endpoint-ashu-761-3-aa24c0d5-7i3yxzspbq-ew.a.run.app/cairo_run


HTTPError: 503 Server Error: Service Unavailable for url: https://endpoint-ashu-761-3-aa24c0d5-7i3yxzspbq-ew.a.run.app/cairo_run

In [110]:
!giza endpoints logs -e {329}

[giza][2024-06-09 13:07:13.222] Getting logs for endpoint 329 ✅ 
2024-06-09T06:19:50.134531Z  INFO orion_runner: ✅ Sierra program downloaded successfully!
2024-06-09T06:19:50.134607Z  INFO orion_runner: 🚀 Server running on 0.0.0.0:8080
Default STARTUP TCP probe succeeded after 1 attempt for container "orion-runner-1" on port 8080.
The request failed because either the HTTP response was malformed or connection to the instance had an error. Additional troubleshooting documentation can be found at: https://cloud.google.com/run/docs/troubleshooting#malformed-response-or-connection-error
thread 'tokio-runtime-worker' panicked at crates/cairo_runner/src/utils.rs:16:78:
called `Result::unwrap()` on an `Err` value: FromStrError
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace
2024-06-09T06:20:23.304933Z  INFO orion_runner: ✅ Sierra program downloaded successfully!
2024-06-09T06:20:23.304999Z  INFO orion_runner: 🚀 Server running on 0.0.0.0:8080
Default STARTUP TCP p